# Data Gathering

### Binary Classification Data

 This data is cat and dogs image data.
 [More details in this link](https://www.kaggle.com/tongpython/cat-and-dog)

In [ ]:
! gdown --id 1akIlxsuWUOR4Kqnl2Zv2t80fK1lRZinL  # Downloading data from google drive using file id . This will work in any colab notebook

In [4]:
! unzip /content/data.zip #Unzipping the zip file

### Multiclass Classification Data

Not found yet

# Importing Libraries

### General libraries

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob

### Model Specific libraries

In [2]:
# import the libraries as shown below
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential

# Data Check

### Data Folder Check

In [12]:
dir_path ="/content/Cat_dog"
print("Folder content",end=" ")
os.listdir(dir_path)

Folder content 

['test', 'train']

### Train data check

In [10]:
dir_path ="/content/Cat_dog/train"
content = os.listdir(dir_path)
print("Categories :",content)
for i in os.listdir(dir_path):
    print("No of Images in ",i," category is ",len(os.listdir(os.path.join(dir_path,i))))

['cats', 'dogs']
No of Images in  cats  category is  4000
No of Images in  dogs  category is  4005


### Test data check

In [11]:
dir_path ="/content/Cat_dog/test"
content = os.listdir(dir_path)
print("Categories :",content)
for i in os.listdir(dir_path):
    print("No of Images in ",i," category is ",len(os.listdir(os.path.join(dir_path,i))))

Categories : ['cats', 'dogs']
No of Images in  cats  category is  1012
No of Images in  dogs  category is  1013


# Model Creation

### Model Creation Inputs

In [19]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
# no. of classes
cl = 2

### VGG 16 weights download

In [20]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

### Adding layers to the vgg16 layers

In [21]:
# our layers - you can add more if you want
Dense1 = Dense(500,name="Dense1",activation='relu')(vgg16.output)
Dense2 = Dense(500,name="Dense2",activation='relu')(Dense1)
x=Flatten()(Dense2)
output = Dense(cl, name="Output", activation='softmax')(x)

### Setting layers training status

In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

### Creating model

In [22]:
# create a model object
model = Model(inputs=vgg16.input, outputs=output)

Visulaizing model 

In [23]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

# Model Compilation

In [24]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Data Preprocessing

In [25]:
train_path = '/content/Cat_dog/train'
test_path = '/content/Cat_dog/test'

In [26]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [32]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 256,
                                                 class_mode = 'categorical')

Found 8005 images belonging to 2 classes.


In [28]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 1,
                                            class_mode = 'categorical')

Found 2023 images belonging to 2 classes.


# Model fitting

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100


#Model Metrics Plotting

In [ ]:
# plot the loss
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')